In [3]:
import pymysql
import pandas as pd
from getpass import getpass

connection = pymysql.connect(
    host="localhost",
    user="root",
    password=getpass("Enter your password: "),
    database="projects"
)
print("Connection Successful")

Connection Successful


1. Calculate daily material efficiency (%) for each machine

In [4]:


mycursor = connection.cursor()

sql = """SELECT Machine_ID, Date,
       CASE 
           WHEN (Material_Used_kg - Material_Waste_kg) > 0 
           THEN ROUND((Product_Output_Units / (Material_Used_kg - Material_Waste_kg)) * 100, 2)
           ELSE NULL 
       END AS Material_Efficiency
FROM sustainable_manufacturing
WHERE Material_Used_kg > 0;

"""

mycursor.execute(sql)
myresult = mycursor.fetchall()

# Process results
temp = []
for data in myresult:
    temp.append(data)

# Create DataFrame
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(temp, columns=columns)
print(df)

# Close resources
# mycursor.close()
# connection.close()

     Machine_ID        Date Material_Efficiency
0          M001  01.01.2020              100.00
1          M001  01.01.2020              100.00
2          M001  01.01.2020              100.00
3          M001  01.01.2020              100.00
4          M001  01.01.2020              100.00
...         ...         ...                 ...
9995       M010  10.01.2020               99.48
9996       M010  10.01.2020               99.48
9997       M010  10.01.2020               99.48
9998       M010  10.01.2020               99.48
9999       M010  10.01.2020               99.48

[10000 rows x 3 columns]


2. Find machines with energy consumption per output unit > 2kWh/unit

In [5]:

mycursor = connection.cursor()

sql = """WITH energy_stats AS (
    SELECT Machine_ID,
           Energy_Consumption_kWh / NULLIF(Product_Output_Units, 0) AS Energy_Per_Unit
    FROM sustainable_manufacturing
)
SELECT Machine_ID, Energy_Per_Unit
FROM energy_stats 
WHERE Energy_Per_Unit > 0.2;

"""

mycursor.execute(sql)
myresult = mycursor.fetchall()

# Process results
temp = []
for data in myresult:
    temp.append(data)

# Create DataFrame
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(temp, columns=columns)
print(df)


     Machine_ID Energy_Per_Unit
0          M001          0.7111
1          M001          0.7111
2          M001          0.7111
3          M001          0.7111
4          M001          0.7111
...         ...             ...
9995       M010          0.6579
9996       M010          0.6579
9997       M010          0.6579
9998       M010          0.6579
9999       M010          0.6579

[10000 rows x 2 columns]


3. Calculate weekly water savings percentage

In [6]:

mycursor = connection.cursor()

sql = """SELECT Machine_ID, YEAR(Date) AS Year, WEEK(Date) AS Week,
       ROUND((SUM(Water_Recycled_Liters)/SUM(Water_Consumption_Liters)) * 100, 2) AS Water_Savings_Pct
FROM sustainable_manufacturing
GROUP BY Machine_ID, Year, Week;

"""

mycursor.execute(sql)
myresult = mycursor.fetchall()

# Process results
temp = []
for data in myresult:
    temp.append(data)

# Create DataFrame
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(temp, columns=columns)
print(df)


  Machine_ID  Year  Week Water_Savings_Pct
0       M001  2001     2             20.00
1       M002  2002     3             16.67
2       M003  2003     3             20.00
3       M004  2004     3             14.12
4       M005  2005     3             20.00
5       M006  2006     3             19.57
6       M007  2007     2             17.78
7       M008  2008     3             20.00
8       M009  2009     3             20.00
9       M010  2010     3             17.50


4. Rank machines by CO₂ emissions per product unit

In [7]:

mycursor = connection.cursor()

sql = """SELECT Machine_ID,
       RANK() OVER (ORDER BY SUM(CO2_Emission_kg)/NULLIF(SUM(Product_Output_Units),0) DESC) AS Emission_Rank
FROM sustainable_manufacturing
GROUP BY Machine_ID;

"""

mycursor.execute(sql)
myresult = mycursor.fetchall()

# Process results
temp = []
for data in myresult:
    temp.append(data)

# Create DataFrame
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(temp, columns=columns)
print(df)


  Machine_ID  Emission_Rank
0       M003              1
1       M006              2
2       M002              3
3       M008              4
4       M005              5
5       M001              6
6       M007              6
7       M009              6
8       M004              9
9       M010             10


5. Identify machines where material waste exceeds 15% of total material used

In [8]:

mycursor = connection.cursor()

sql = """WITH waste AS (
				SELECT machine_id, CAST(SUM(material_waste_kg) AS float) AS total_waste,
                0.10 * (SELECT SUM(material_waste_kg) FROM sustainable_manufacturing) AS ten_percent_waste
                FROM sustainable_manufacturing
                GROUP BY machine_id)

SELECT machine_id, total_waste, ten_percent_waste
FROM waste
WHERE total_waste > 0.10 * (SELECT SUM(material_waste_kg) FROM sustainable_manufacturing)
;

"""

mycursor.execute(sql)
myresult = mycursor.fetchall()

# Process results
temp = []
for data in myresult:
    temp.append(data)

# Create DataFrame
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(temp, columns=columns)
print(df)


  machine_id  total_waste ten_percent_waste
0       M001      50000.0          45500.00
1       M005      55000.0          45500.00
2       M008      60000.0          45500.00
3       M009      50000.0          45500.00


6. Compare machines' productivity changes between first and last weeks of January 2020

In [13]:

mycursor = connection.cursor()

sql = """WITH first_week_tbl AS (
SELECT machine_id,
		AVG(energy_consumption_kwh / product_output_units) * 100 AS energy_productivity
FROM sustainable_manufacturing
WHERE WEEK(date, 3) = (SELECT WEEK("2020-01-01", 3) AS firstweek)
GROUP BY machine_id),

	last_week_tbl AS (

SELECT machine_id,
		AVG(energy_consumption_kwh / product_output_units) * 100 AS energy_productivity
FROM sustainable_manufacturing
WHERE WEEK(date, 3) = (SELECT WEEK("2020-01-07", 3) AS lastweek)
GROUP BY machine_id)

SELECT first_week_tbl.machine_id AS machine_id, first_week_tbl.energy_productivity AS energy_productivity
FROM first_week_tbl

UNION 

SELECT last_week_tbl.machine_id AS machine_id, last_week_tbl.energy_productivity AS energy_productivity
FROM last_week_tbl
;

"""

mycursor.execute(sql)
myresult = mycursor.fetchall()

# Process results
temp = []
for data in myresult:
    temp.append(data)

# Create DataFrame
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(temp, columns=columns)
print(df)


Empty DataFrame
Columns: [machine_id, energy_productivity]
Index: []


7. Compare average energy consumption on weekends vs. weekdays (assuming 5-day work week).

In [16]:

mycursor = connection.cursor()

sql = """SELECT "Weekday" AS Day, AVG(energy_consumption_kwh) AS Energy_consumption
FROM sustainable_manufacturing
WHERE WEEKDAY(date) IN (0,4)

UNION

SELECT "Weekend", AVG(energy_consumption_kwh)
FROM sustainable_manufacturing
WHERE WEEKDAY(date) IN (5,6)
;
"""

mycursor.execute(sql)
myresult = mycursor.fetchall()

# Process results
temp = []
for data in myresult:
    temp.append(data)

# Create DataFrame
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(temp, columns=columns)
print(df)


       Day Energy_consumption
0  Weekday           302.5000
1  Weekend           303.7500


8. List machines performing below department average in both energy efficiency and output.

In [17]:

mycursor = connection.cursor()

sql = """WITH department_metrics AS (
  SELECT 
    AVG(Product_Output_Units) AS dept_avg_output,
    AVG(Product_Output_Units/NULLIF(Energy_Consumption_kWh,0)) AS dept_avg_energy_eff
  FROM sustainable_manufacturing
),
machine_metrics AS (
  SELECT 
    Machine_ID,
    AVG(Product_Output_Units) AS machine_output,
    AVG(Product_Output_Units/NULLIF(Energy_Consumption_kWh,0)) AS machine_energy_eff
  FROM sustainable_manufacturing
  GROUP BY Machine_ID
)
SELECT m.Machine_ID
FROM machine_metrics m
CROSS JOIN department_metrics d
WHERE m.machine_output < d.dept_avg_output
  AND m.machine_energy_eff < d.dept_avg_energy_eff;

"""

mycursor.execute(sql)
myresult = mycursor.fetchall()

# Process results
temp = []
for data in myresult:
    temp.append(data)

# Create DataFrame
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(temp, columns=columns)
print(df)


  Machine_ID
0       M002
1       M006
2       M007


9. Find machines with top 10% productivity but bottom 25% environmental performance.

In [18]:

mycursor = connection.cursor()

sql = """WITH productivity AS (
  SELECT Machine_ID,
    PERCENT_RANK() OVER (ORDER BY SUM(Product_Output_Units) DESC) AS prod_rank
  FROM sustainable_manufacturing
  GROUP BY Machine_ID
),
environment AS (
  SELECT Machine_ID,
    PERCENT_RANK() OVER (ORDER BY SUM(CO2_Emission_kg) + SUM(Energy_Consumption_kWh) ASC) AS env_rank
  FROM sustainable_manufacturing
  GROUP BY Machine_ID
)
SELECT p.Machine_ID
FROM productivity p
JOIN environment e USING (Machine_ID)
WHERE p.prod_rank <= 0.10
  AND e.env_rank >= 0.75;


"""

mycursor.execute(sql)
myresult = mycursor.fetchall()

# Process results
temp = []
for data in myresult:
    temp.append(data)

# Create DataFrame
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(temp, columns=columns)
print(df)


  Machine_ID
0       M008


10. Flag days where a machine's output deviates >2σ from its 7-day moving average.

In [19]:

mycursor = connection.cursor()

sql = """WITH moving_metrics AS (
  SELECT Machine_ID, Date, Product_Output_Units,
    AVG(Product_Output_Units) OVER (
      PARTITION BY Machine_ID 
      ORDER BY Date 
      ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ) AS ma_7d,
    STDDEV(Product_Output_Units) OVER (
      PARTITION BY Machine_ID 
      ORDER BY Date 
      ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ) AS std_7d
  FROM sustainable_manufacturing
)
SELECT Machine_ID, Date, Product_Output_Units,
  ROUND((Product_Output_Units - ma_7d)/NULLIF(std_7d,0), 2) AS z_score
FROM moving_metrics
WHERE ABS((Product_Output_Units - ma_7d)/NULLIF(std_7d,0)) > 2;


"""

mycursor.execute(sql)
myresult = mycursor.fetchall()

# Process results
temp = []
for data in myresult:
    temp.append(data)

# Create DataFrame
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(temp, columns=columns)
print(df)


Empty DataFrame
Columns: [Machine_ID, Date, Product_Output_Units, z_score]
Index: []


11. Detect days with simultaneous 30%+ spikes in energy/water use and CO₂ emissions.

In [20]:

mycursor = connection.cursor()

sql = """WITH resource_spikes AS (
  SELECT Machine_ID, Date,
    CO2_Emission_kg / NULLIF(LAG(CO2_Emission_kg, 1) OVER w, 0) - 1 AS co2_increase,
    Energy_Consumption_kWh / NULLIF(LAG(Energy_Consumption_kWh, 1) OVER w, 0) - 1 AS energy_increase,
    Water_Consumption_Liters / NULLIF(LAG(Water_Consumption_Liters, 1) OVER w, 0) - 1 AS water_increase
  FROM sustainable_manufacturing
  WINDOW w AS (PARTITION BY Machine_ID ORDER BY Date)
)
SELECT Machine_ID, Date,
  ROUND(co2_increase*100,2) AS co2_pct,
  ROUND(GREATEST(energy_increase, water_increase)*100,2) AS resource_pct
FROM resource_spikes
WHERE co2_increase >= 0.3
  AND (energy_increase >= 0.3 OR water_increase >= 0.3);


"""

mycursor.execute(sql)
myresult = mycursor.fetchall()

# Process results
temp = []
for data in myresult:
    temp.append(data)

# Create DataFrame
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(temp, columns=columns)
print(df)


Empty DataFrame
Columns: [Machine_ID, Date, co2_pct, resource_pct]
Index: []


12. Create composite score weighting: 40% emissions, 30% energy, 20% water, 10% waste"

In [21]:

mycursor = connection.cursor()

sql = """WITH normalized AS (
    SELECT Machine_ID,
           (CO2_Emission_kg - MIN(CO2_Emission_kg) OVER()) / 
           (MAX(CO2_Emission_kg) OVER() - MIN(CO2_Emission_kg) OVER()) AS norm_co2,
           (Energy_Consumption_kWh - MIN(Energy_Consumption_kWh) OVER()) / 
           (MAX(Energy_Consumption_kWh) OVER() - MIN(Energy_Consumption_kWh) OVER()) AS norm_energy,
           (Water_Consumption_Liters - MIN(Water_Consumption_Liters) OVER()) / 
           (MAX(Water_Consumption_Liters) OVER() - MIN(Water_Consumption_Liters) OVER()) AS norm_water,
           (Material_Waste_kg - MIN(Material_Waste_kg) OVER()) / 
           (MAX(Material_Waste_kg) OVER() - MIN(Material_Waste_kg) OVER()) AS norm_waste
    FROM sustainable_manufacturing
)
SELECT Machine_ID,
       (0.4 * norm_co2 + 0.3 * norm_energy + 0.2 * norm_water + 0.1 * norm_waste) AS Sustainability_Index
FROM normalized;



"""

mycursor.execute(sql)
myresult = mycursor.fetchall()

# Process results
temp = []
for data in myresult:
    temp.append(data)

# Create DataFrame
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(temp, columns=columns)
print(df)


     Machine_ID Sustainability_Index
0          M001              0.77584
1          M001              0.77584
2          M001              0.77584
3          M001              0.77584
4          M001              0.77584
...         ...                  ...
9995       M010              0.01200
9996       M010              0.01200
9997       M010              0.01200
9998       M010              0.01200
9999       M010              0.01200

[10000 rows x 2 columns]
